In [1]:
PATH_DIR_PREDICT= r"C:\project\Computer_Vision\term_paper_2023\записи\anti UAV\yolo_result_pred\detect\predict\labels"
PATH_DIR_TRUE = r"C:\project\Computer_Vision\term_paper_2023\записи\anti UAV\Anti_UAV_yolo\test\labels"

In [2]:
from PIL import Image
import os
import pandas as pd
import glob
import shutil
from tqdm.auto import tqdm
from utility import get_iou_yolo

In [3]:
paths_predict = glob.glob(os.path.join(PATH_DIR_PREDICT,"*.txt"))
paths_true = glob.glob(os.path.join(PATH_DIR_TRUE,"*.txt"))

In [4]:
df = pd.DataFrame (paths_true, columns=['path_label'])
df["id"] = df['path_label'].apply(lambda x : x.split('\\')[-1].split('.')[0])
df["path_predict"] = df['id'].apply(lambda id : os.path.join(PATH_DIR_PREDICT,f"{id}.txt") )

In [39]:
accuracy=[]
threshold_iou = 0.25
part_area_drone = []

for i, row in tqdm(df.iterrows()):

    with open(row['path_label']) as f:
        coords_true = f.read().split("\n")
        coords_true.remove('')

    if not os.path.isfile(row["path_predict"]):
        accuracy += [False]*len(coords_true)
        for coords_true_ in coords_true:
            coords_true_ = coords_true_.split(" ")
            part_area_drone.append(float(coords_true_[3])*float(coords_true_[4]))
        continue

    with open(row['path_predict']) as f:
        coords_predict =  f.read().split("\n")
        coords_predict.remove('')

    # print(coords_true , coords_predict)
    for coords_true_ in coords_true:
        coords_true_ = coords_true_.split(" ")
        part_area_drone.append(float(coords_true_[3])*float(coords_true_[4]))
        status = False
        for coords_predict_ in coords_predict:
            coords_predict_ = coords_predict_.split(" ")
            iou = get_iou_yolo(coords_true_[1:],coords_predict_[1:])
            if iou>threshold_iou:
                status = True
                break
        accuracy.append(status)

print(accuracy)




0it [00:00, ?it/s]

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 

In [38]:
sum(accuracy)/len(accuracy)

0.8926503340757238

In [41]:

df_analysis = pd.DataFrame (part_area_drone, columns=['part_area_drone'])
df_analysis["accuracy"] = accuracy


In [48]:
df_analysis[df_analysis["accuracy"] == False]["part_area_drone"].mean()

0.002977604146656166

In [49]:
df_analysis[df_analysis["accuracy"] == True]["part_area_drone"].mean()

0.014916465252105882